# Flowers CNN - SOFTMAX - EfficientNet

**Description:** Classify cats and dogs with a simple Convolutional Network<br>
                This version is programmed as a multiclass classifier with a softmax function in last layer <br>
                This version uses EfficientNet a very large (and efficient) network that is the market standard <br>
                 
**Dataset:** Tensorflow Flowers dataset 5 classes
             

In [ ]:
%matplotlib inline
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' # to avoid warning messages


import matplotlib.pyplot as plt
import matplotlib.image as mpimg

import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import array_to_img, img_to_array, load_img
from tensorflow.keras import Input
from tensorflow.keras.layers import ReLU, Dense, Softmax, Rescaling, Flatten, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model, Sequential
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras import Model
from tensorflow.keras.optimizers import RMSprop, Adam
from tensorflow.keras.utils import image_dataset_from_directory
import tensorflow as tf

import numpy as np

import sys
sys.stderr = open('err.txt', 'w')

AUGMENTATION = False

#### **Identifying GPU to use**

In [ ]:
from tensorflow.python.client import device_lib
devices = device_lib.list_local_devices()
gpu_devices = [device for device in devices if device.device_type == 'GPU']
for gpu in gpu_devices:
    print('Using', gpu.physical_device_desc)

In [ ]:
#### Hyperparameters
training_epochs = 50
lr = 1e-5
img_height = 224
img_width = 224

In [ ]:
import tensorflow_datasets as tfds 
ds, ds_info = tfds.load(
    'tf_flowers',
    split='train',
    with_info=True,
    download=False
)

In [ ]:
for example in ds.take(4):
    print("Shape:", example['image'].shape, "Label:", example['label'])

#### Visualize Flowers

In [ ]:
ROWS = 4
COLS = 4
plt.figure(figsize=(8,8))
for i, example in enumerate(ds.take(ROWS*COLS)):
    image = example['image']
    label = example['label']
    name = ds_info.features['label'].int2str(label)
    plt.subplot(ROWS, COLS, i+1)
    plt.title("{} ({})".format(name, label))
    plt.axis('off')
    plt.imshow(image)

In [ ]:
ds_train_, ds_valid_ = tfds.load(
    'tf_flowers',
    as_supervised=True,
    split=['train[:75%]', 'train[75%:]'],
)

In [ ]:
SIZE = [img_height, img_width]

def preprocess(image, label):
    image = tf.image.resize(image, size=SIZE)
    image = tf.image.convert_image_dtype(image, dtype=tf.float32)
    return image, label

def augment(image, label):
    image = tf.image.random_flip_left_right(image)
    return image, label

In [ ]:
AUTO = tf.data.experimental.AUTOTUNE # TensorFlow can automatically apply optimizations to some parts of the pipeline
NUM_TRAINING_IMAGES = ds_info.splits['train'].num_examples
SHUFFLE_BUFFER =  NUM_TRAINING_IMAGES // 4
BATCH_SIZE = 32

train_dataset = (ds_train_
            .map(preprocess, AUTO) # do any (non-random) preprocessing first
            .cache() # and then keep the images in memory cache
            .shuffle(SHUFFLE_BUFFER) # randomize image order while training
            .batch(BATCH_SIZE)
            .map(augment, AUTO) # put (random) augmentation after batching
            .prefetch(AUTO) # use CPU to load data while TPU is working
)

validation_dataset = (ds_valid_
            .map(preprocess, AUTO)
            .batch(BATCH_SIZE)
            .cache()
            .prefetch(AUTO)
)

#### **Network Architecture Definition**

In [ ]:
from keras.applications import EfficientNetB7
model = Sequential()

efnModel = EfficientNetB7(weights = 'imagenet', 
                          input_shape = (img_height, img_width, 3), 
                       include_top = False)
                          
>>>>> WRITE HERE YOUR CODE EfficientNet or VGG
>>>>> Carful with the loss use sparse categorical as it is encoded like that

# decay is included for backward compatibility to allow time inverse decay of lr
opt1 = RMSprop(learning_rate=lr, decay=1e-6)
opt2 = Adam(learning_rate=lr) 

model.compile(loss='sparse_categorical_crossentropy',
              optimizer = opt1, 
              metrics = ['acc'])

model.summary()


In [ ]:
#Setting callbakcs

initial_learning_rate = 0.015
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate, decay_steps=100000, decay_rate=0.96, staircase=True
)

early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=5,
    min_delta=0.0000001,
    restore_best_weights=True,
)

plateau = ReduceLROnPlateau(
    monitor='val_loss',
    factor = 0.2,                                     
    patience = 10,                                   
    min_delt = 0.0000001,                                
    cooldown = 0,                               
    verbose = 1
) 


early_stopping = tf.keras.callbacks.EarlyStopping(
    patience=10, restore_best_weights=True
)

In [ ]:
history = model.fit(train_dataset, epochs=training_epochs, validation_data=validation_dataset, callbacks = [early_stopping])

#### **Evaluate the Results**

In [ ]:
# Plot training loss, accuracy
plt.figure(figsize=(12, 5))

# Plot loss
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Training Loss', color='orange')
plt.plot(history.history['val_loss'], label='Validation Loss', color='blue')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.ylim(0,1)
plt.title('Training and Validation Loss')
plt.legend()

# Plot accuracy
plt.subplot(1, 2, 2)
plt.plot(history.history['acc'], label='Training Accuracy')
plt.plot(history.history['val_acc'], label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy (%)')
plt.title('Training and Validation Accuracy')
plt.ylim(0,1)
plt.legend()
plt.show()


#### **Generate Confusion Matrix**

In [ ]:
# Generate predictions for the validation dataset
y_true = []
y_pred = []

for images, labels in validation_dataset:
    y_true.extend(labels.numpy())  # True labels 
    preds = model.predict(images, verbose=0)  # Model predictions
#    y_pred.extend((preds > 0.5).astype(int))   This is for binary
    y_pred.extend(np.argmax(preds, axis = 1))

y_true = np.argmax(y_true, axis = 1)
y_pred = np.array(y_pred)

print(y_true.shape, y_pred.shape)

In [ ]:
# Compute confusion matrix
cm = confusion_matrix(y_true, y_pred)

# Display confusion matrix
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=class_names)
disp.plot(cmap=plt.cm.Blues, xticks_rotation="vertical")
plt.title("Confusion Matrix CNN Binary Classifier")
plt.show()

In [ ]:
# Final accuracy
final_accuracy = history.history['acc'][-1]  # Last epoch training accuracy
final_val_accuracy = history.history['val_acc'][-1] 

print(f"Final Training Accuracy: {final_accuracy}")
print(f"Final Validation Accuracy: {final_val_accuracy}")

In [ ]:
import session_info
session_info.show(html=False)